In [229]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [284]:
%reload_ext autoreload

In [239]:
import json

with open('frontend_sync_20.txt', 'rb') as f:
    request = json.load(f)

In [240]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T13:56:04.897505Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T13:56:04.898545Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T13:56:04.899534Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T13:56:04.900492Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T13:56:04.901471Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T13:56:04.902347Z", "msg": "getting feature vector from mind service"}
{"level": 

KeyboardInterrupt: 

In [ ]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-07-04T12:15:14Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")


In [ ]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [92]:
pim_result = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqy88qzb19jqz5prjfr76y"}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})

In [95]:
group

{'group': {'15': [{'id': '1bc499f637aa41779390460a565e4524',
    'originalText': 'So you can see the that areas you have five alias mentioned here.',
    'confidence': 0.9471649,
    'startTime': '2019-09-22T09:54:05Z',
    'endTime': '2019-09-22T09:54:12Z',
    'duration': 7,
    'recordingId': '39705b73c1074b96822bf145c23e7704',
    'spokenBy': '31a3ba4761854ad9a041ddf1c4c6a1dc',
    'languageCode': 'en-IN',
    'transcriber': 'deepgram',
    'status': 'completed',
    'transcriptId': '86dfd87a-6a7d-4164-ba01-a40a24c8daef',
    'createdAt': '2019-09-22T09:54:12.942977446Z',
    'updatedAt': '2019-09-22T09:54:14.715094811Z',
    'deletedAt': None,
    'deleted': False}],
  '24': [{'id': '8ad2cdf53c0c47ee97ecc9eb649af405',
    'originalText': "Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey thr

In [96]:
topic_pim = {
}
group_result = {}
for keys in group['group'].keys():
    for seg in group['group'][keys]:
        group_result[seg['recordingId']] = keys

In [118]:
ranked_pims = sorted([(k,v) for (k,v) in pim_result.items()], key= lambda kv: kv[1])

In [119]:
used_topics = []
group_no = None
index = 0
for (rec_id, distance) in ranked_pims:
    if rec_id in group_result.keys():
        group_no = group_result[rec_id]
        
        if group_no not in used_topics:
            topic_pim[index] = group_no
            used_topics.append(group_no)
            index += 1

In [120]:
topic_pim

{0: '24', 1: '40', 2: '32', 3: '15'}

In [125]:
final_output = []
final_output = list(map(lambda x: group['group'][x] , topic_pim.values()))

In [126]:
final_output[0][0]

{'id': '8ad2cdf53c0c47ee97ecc9eb649af405',
 'originalText': "Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey through the sixteenth end of the UX design process that before starting any successful uX project you'll need to be in tune with two things your user in your brain. I start with the user big question around this is what problems are they making once you can identify the problem, then you can begin to start working to towards the solution second is understanding the brain and how does the project you're are working on aligned with the branch mission and goal to summarize in this stage you wanna identify the problems facing the users as well as the core goals of the brand and see how they along as a designer you probably have a gun of a*sumptions about what the user wants or doesn't want b

In [129]:
users = []
for result in final_output:
    temp_users = []
    for seg in result:
        if seg['spokenBy'] not in temp_users:
            temp_users.append(seg['spokenBy'])
    users.append(temp_users)

In [130]:
users

[['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc']]

# Testing topic level pims

## Read json Request

In [270]:
import json

with open('eng_sync_24.txt', 'rb') as f:
    request = json.load(f)

## Get Groups for the request

In [271]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T15:10:03.031642Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T15:10:03.032400Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T15:10:03.033067Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T15:10:03.034103Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T15:10:03.034664Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-25T15:10:03.035264Z", "msg": "getting feature vector from mind service"}
{"level": 

{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-09-25T15:11:20.671076Z", "msg": "Response Recieved"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-09-25T15:11:20.671635Z", "msg": "Response Recieved"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-09-25T15:11:20.672503Z", "msg": "Response Recieved"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-09-25T15:11:20.673045Z", "msg": "Response Recieved"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-09-25T15:11:20.673586Z", "msg": "Response Recieved"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-09-25T15:11:20.674065Z", "msg": "Response Recieved"}
{"level": "info", "filename": "scorer.py", "lineno": 32, "module": "scorer", "ts": "2019-09-25T15:11:20.674601Z", "msg": "Response Recieved"}
{"leve

{"level": "info", "filename": "grouper_segments.py", "lineno": 120, "module": "grouper_segments", "v is : ": 0.15, "ts": "2019-09-25T15:11:23.208018Z", "msg": "pruning value"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 120, "module": "grouper_segments", "v is : ": 0.15, "ts": "2019-09-25T15:11:23.208561Z", "msg": "pruning value"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 120, "module": "grouper_segments", "v is : ": 0.15, "ts": "2019-09-25T15:11:23.210230Z", "msg": "pruning value"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 120, "module": "grouper_segments", "v is : ": 0.15, "ts": "2019-09-25T15:11:23.210943Z", "msg": "pruning value"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 120, "module": "grouper_segments", "v is : ": 0.15, "ts": "2019-09-25T15:11:23.211654Z", "msg": "pruning value"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 120, "module": "grouper_segments", "v is : ": 0.15, "ts":

{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.23198573800994446, "ts": "2019-09-25T15:11:23.491572Z", "msg": "Meeting Graph results"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.23198573800994446, "ts": "2019-09-25T15:11:23.492203Z", "msg": "Meeting Graph results"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.23198573800994446, "ts": "2019-09-25T15:11:23.492806Z", "msg": "Meeting Graph results"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.23198573800994446, "ts": "20

{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.2318867819552608, "ts": "2019-09-25T15:11:23.805634Z", "msg": "Meeting Graph results"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.2318867819552608, "ts": "2019-09-25T15:11:23.806319Z", "msg": "Meeting Graph results"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.2318867819552608, "ts": "2019-09-25T15:11:23.807004Z", "msg": "Meeting Graph results"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 389, "module": "grouper_segments", "edges before prunning": 14706, "edges after prunning": 2207, "modularity ": 0.2318867819552608, "ts": "2019-0

...

{"level": "info", "filename": "grouper_segments.py", "lineno": 414, "module": "grouper_segments", "PIMs": {"2": {"segment0": [["Because last time Taiwan one got yes."], "2019-09-24T06:24:53Z", "1a21542584494fcaba957d768b595b80", "43f9c63b69684e788694b5531eab4560"]}, "4": {"segment0": [["But it was not working that that is I one."], "2019-09-24T06:32:51Z", "1a21542584494fcaba957d768b595b80", "f201cfea173f4d7a9b5701edd2c62791"], "segment1": [["The the current call that trying always we need to get the event from."], "2019-09-24T06:32:58Z", "1a21542584494fcaba957d768b595b80", "1878faea1939454498c9550e2aa4a529"], "segment2": [["We can got ads we have the that they development can add a CSS source to either does something how like the batch up the way that is there. But the main is the setting the audio stopping not nothing we should the approach this is a mute audio from there that is one option or the window there is a way to move the so."], "2019-09-24T06:33:13Z", "1a21542584494fcaba957d

{"level": "info", "filename": "grouper_segments.py", "lineno": 414, "module": "grouper_segments", "PIMs": {"2": {"segment0": [["Because last time Taiwan one got yes."], "2019-09-24T06:24:53Z", "1a21542584494fcaba957d768b595b80", "43f9c63b69684e788694b5531eab4560"]}, "4": {"segment0": [["But it was not working that that is I one."], "2019-09-24T06:32:51Z", "1a21542584494fcaba957d768b595b80", "f201cfea173f4d7a9b5701edd2c62791"], "segment1": [["The the current call that trying always we need to get the event from."], "2019-09-24T06:32:58Z", "1a21542584494fcaba957d768b595b80", "1878faea1939454498c9550e2aa4a529"], "segment2": [["We can got ads we have the that they development can add a CSS source to either does something how like the batch up the way that is there. But the main is the setting the audio stopping not nothing we should the approach this is a mute audio from there that is one option or the window there is a way to move the so."], "2019-09-24T06:33:13Z", "1a21542584494fcaba957d

{"level": "info", "filename": "grouper_segments.py", "lineno": 414, "module": "grouper_segments", "PIMs": {"2": {"segment0": [["Because last time Taiwan one got yes."], "2019-09-24T06:24:53Z", "1a21542584494fcaba957d768b595b80", "43f9c63b69684e788694b5531eab4560"]}, "4": {"segment0": [["But it was not working that that is I one."], "2019-09-24T06:32:51Z", "1a21542584494fcaba957d768b595b80", "f201cfea173f4d7a9b5701edd2c62791"], "segment1": [["The the current call that trying always we need to get the event from."], "2019-09-24T06:32:58Z", "1a21542584494fcaba957d768b595b80", "1878faea1939454498c9550e2aa4a529"], "segment2": [["We can got ads we have the that they development can add a CSS source to either does something how like the batch up the way that is there. But the main is the setting the audio stopping not nothing we should the approach this is a mute audio from there that is one option or the window there is a way to move the so."], "2019-09-24T06:33:13Z", "1a21542584494fcaba957d

{"level": "info", "filename": "grouper_segments.py", "lineno": 414, "module": "grouper_segments", "PIMs": {"2": {"segment0": [["Because last time Taiwan one got yes."], "2019-09-24T06:24:53Z", "1a21542584494fcaba957d768b595b80", "43f9c63b69684e788694b5531eab4560"]}, "4": {"segment0": [["But it was not working that that is I one."], "2019-09-24T06:32:51Z", "1a21542584494fcaba957d768b595b80", "f201cfea173f4d7a9b5701edd2c62791"], "segment1": [["The the current call that trying always we need to get the event from."], "2019-09-24T06:32:58Z", "1a21542584494fcaba957d768b595b80", "1878faea1939454498c9550e2aa4a529"], "segment2": [["We can got ads we have the that they development can add a CSS source to either does something how like the batch up the way that is there. But the main is the setting the audio stopping not nothing we should the approach this is a mute audio from there that is one option or the window there is a way to move the so."], "2019-09-24T06:33:13Z", "1a21542584494fcaba957d

{"level": "info", "filename": "grouper_segments.py", "lineno": 414, "module": "grouper_segments", "PIMs": {"2": {"segment0": [["Because last time Taiwan one got yes."], "2019-09-24T06:24:53Z", "1a21542584494fcaba957d768b595b80", "43f9c63b69684e788694b5531eab4560"]}, "4": {"segment0": [["But it was not working that that is I one."], "2019-09-24T06:32:51Z", "1a21542584494fcaba957d768b595b80", "f201cfea173f4d7a9b5701edd2c62791"], "segment1": [["The the current call that trying always we need to get the event from."], "2019-09-24T06:32:58Z", "1a21542584494fcaba957d768b595b80", "1878faea1939454498c9550e2aa4a529"], "segment2": [["We can got ads we have the that they development can add a CSS source to either does something how like the batch up the way that is there. But the main is the setting the audio stopping not nothing we should the approach this is a mute audio from there that is one option or the window there is a way to move the so."], "2019-09-24T06:33:13Z", "1a21542584494fcaba957d

In [272]:
request

{'body': {'contextId': '01DBB3SN874B4V18DCP4ATMRXA',
  'mindId': '01DAAYHEKY5F4E02QVRJPTFTXV',
  'instanceId': 'b5251484-156f-459f-bd43-fe32dfcf5f26',
  'segments': [{'id': 'c2f875ac67744ac3a2c28e906c724ecb',
    'originalText': "Yesterday I added I'll deploy the conflict config with everything production.",
    'confidence': 0.8851404,
    'startTime': '2019-09-24T06:14:14Z',
    'endTime': '2019-09-24T06:14:23Z',
    'duration': 9,
    'recordingId': 'fe4e5757f6f74613923001e750406a14',
    'spokenBy': '1a21542584494fcaba957d768b595b80',
    'languageCode': 'en-IN',
    'transcriber': 'deepgram',
    'status': 'completed',
    'transcriptId': '759983a5-0d7a-44f0-8667-2ffd7772f3f6',
    'createdAt': '2019-09-24T06:14:23.659120305Z',
    'updatedAt': '2019-09-24T06:14:25.399576139Z',
    'deletedAt': None,
    'deleted': False},
   {'id': '1979a07edea74a9aaec7b96ac8942c96',
    'originalText': 'We.',
    'confidence': 0.30408967,
    'startTime': '2019-09-24T06:17:50Z',
    'endTime': '

## Get PIMs for the request

In [273]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [274]:
pim_result = {}
pim_response = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01DAAYHEKY5F4E02QVRJPTFTXV"}
pim_response = {"contextId": request["body"]["contextId"], "mindId": "01DAAYHEKY5F4E02QVRJPTFTXV", "segments": []}
temp = {}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})
    temp = seg
    temp["distance"] = pim_result[seg["recordingId"]]
    pim_response["segments"].append(temp)

## Extract topic level pims

In [286]:
import sys
sys.path.append("../")

import extract_topic_pims

In [287]:
from extract_topic_pims.main import handler

res = handler({"body":{"groups": group["group"], "pims": pim_response}}, None)
final_pims = json.loads(res)["body"]

{'de0a2633e87840609f871248abb346e5': '2', '99a048185e7b43f1ab9865c90fd291ab': '4', '3e6ba0d9560c4c98aac0a18796383124': '4', 'cc7f46db07ae4c20b90faa2bd3b84d4f': '4', '2ab3610e78114cdb9d3ed967fe06f166': '4', 'ebe6026c5c944c1a88f32c5a1e16d4d6': '7', '5d11c24156644c1993575c83f77828f7': '7', 'b2285149eef24fd791aeacba5fd26ce6': '7', '69c9cb0fec7f4c8381038e06d72aa2b6': '7', '6f659fc624c34dd195ff3ca512fa5da3': '7', '6be58110162141eea436f40f26426133': '7', 'eba0605e90cf47798eb1f7f3cfb769f4': '7', '631c5344a7c1426c80ad7280ee646870': '7', '2d4f8d7300b74e6ab97e62aedce06293': '7', '7760fe1bf14140268d788cd55ee21f78': '7', '9e37855d22544ae08625d85d45ad5b3c': '7', 'c036272a2eab43079570e510cfeb7f48': '7', '50cb9d2a4f1042e995db811fb9b76a0d': '7', 'c52d1790ed094abebbd660bb8a686233': '7', 'da0282fd3814434c9642f9c8e83dfca1': '8', '99aad766868e427d8eb6fd9bcf71cdcc': '15', 'd32526b8b2644599897212431d36b65e': '15', 'f148b57efcd54ae29c7edc4d0c5f1204': '15', '6d636a0d4ab042fc8b3b46932fae9266': '15', 'fe4e5757f6

In [288]:
len(group['group']['76'])

1

In [289]:
for groupid in final_pims:
    user_list =[]
    seg_list = []
    keyphrase = []
    for segi in groupid:
        if segi['spokenBy'] not in user_list:
            user_list.append(segi['spokenBy'])
        seg_list.append(segi['originalText'])
        #keyphrase.append(get_desc(segi['originalText']))
    print ("User", end=" ")
    print (*user_list, sep=", ", end=" ")
    print ("Discussed \n\n Text: ", *seg_list, "\n\n")
    print (*get_desc(" ".join(sent for sent in seg_list))[1][:5], sep=", ")
    print ( "\n\n\n\n")

User 65bb83952fb54409a4bb59bb707f1375, 62b6ae1d7f834b0bb2055f7c72bc3368 Discussed 

 Text:  It's like that I I don't think we can handle it and because it suppose only one recording is paused then the record record component. I not wanted and did is not affected through recorder and know events can be listened by the recorder. Right is it possible for us to you video audio from item. It's like in general, we can do it, but it's it's like by it's it will not be our. No for example at a high frame that meet you see great and we you to know what the timeframe pick up this that I stream on the down. We can think guys I'm trying Dont bye. It's it's embedded by saying the the. It two so basically you see on the basically like for example, there's no external that they can control from high frame to top all the you know stumbling video audio from the right. 


example at a high frame control, recorder and know events, video audio, meet you see great, timeframe pick





User 1a21542584494fcab

In [222]:
import sys
sys.path.append("../../../ai-engine/pkg/")

from graphrank.core import GraphRank
from graphrank.utils import GraphUtils, TextPreprocess

gr = GraphRank()
tp = TextPreprocess()
gu = GraphUtils()

def get_desc(sentence):
    original_tokens, pos_tuple, filtered_pos_tuple = tp.preprocess_text(sentence, filter_by_pos=True, stop_words=False)
    word_graph = gr.build_word_graph(graph_obj=None, input_pos_text=pos_tuple, window=4, preserve_common_words=False)
    normal_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, post_process=True)
    desc_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, descriptive=True, post_process_descriptive=True)
    desc_keyphrase = sorted(desc_keyphrase, key=lambda kv:kv[1], reverse=True)
    normal_kp = [phrase for phrase, score in normal_keyphrase]
    desc_kp = [phrase for phrase, score in desc_keyphrase]
    
    return normal_kp, desc_kp